# Simplicits Easy API Demo

[Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/) is a mesh-free, representation-agnostic way to simulate elastic deformations.

In [1]:
# Imports
import builtins
import copy
import threading
from contextlib import contextmanager
from pathlib import Path

import torch
import kaolin as kal
import k3d
from loguru import logger
from tqdm.auto import tqdm
from ipywidgets import Button, HBox, VBox
from IPython.display import display

logger.info("Imports loaded")

2025-09-30 01:41:33.591 | INFO     | __main__:<module>:16 - Imports loaded


In [ ]:
# Helper functions
@contextmanager
def training_progress_hook(total_steps):
    """Hook into Kaolin's training loop to show progress bar."""
    import kaolin.physics.simplicits.easy_api as easy_api
    
    original_range = getattr(easy_api, "range", builtins.range)
    
    def tqdm_range(*args):
        rng = original_range(*args)
        if hasattr(rng, "__len__") and len(rng) == total_steps:
            return tqdm(rng, desc="Training", total=total_steps, unit="step", leave=False)
        return rng
    
    easy_api.range = tqdm_range
    try:
        yield
    finally:
        if original_range is builtins.range:
            if hasattr(easy_api, "range"):
                delattr(easy_api, "range")
        else:
            easy_api.range = original_range


def load_and_sample_mesh(mesh_path, num_samples=200_000):
    """Load mesh, center it, and sample interior points."""
    mesh = kal.io.import_mesh(mesh_path, triangulate=True).cuda()
    mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices.unsqueeze(0), normalize=True).squeeze(0)
    orig_vertices = mesh.vertices.clone()
    logger.info(f"Loaded mesh: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")
    
    # Sample interior points
    min_corner = orig_vertices.min(dim=0).values
    max_corner = orig_vertices.max(dim=0).values
    uniform_pts = torch.rand(num_samples, 3, device='cuda') * (max_corner - min_corner) + min_corner
    inside = kal.ops.mesh.check_sign(
        mesh.vertices.unsqueeze(0), mesh.faces, uniform_pts.unsqueeze(0), hash_resolution=512
    ).squeeze()
    pts = uniform_pts[inside]
    logger.info(f"Sampled {len(pts)} interior points")
    
    return mesh, orig_vertices, pts


def create_sim_object(pts, youngs=1e5, poisson=0.45, density=500.0, volume=0.5, handles=5, steps=10000):
    """Create and train a Simplicits simulation object with progress bar."""
    yms = torch.full((pts.shape[0],), youngs, device="cuda")
    prs = torch.full((pts.shape[0],), poisson, device="cuda")
    rhos = torch.full((pts.shape[0],), density, device="cuda")
    
    logger.info(f"Training Simplicits object: {steps} steps, {handles} handles")
    
    # Use the training progress hook
    with training_progress_hook(steps):
        sim_obj = kal.physics.simplicits.SimplicitsObject.create_trained(
            pts, yms, prs, rhos, volume,
            num_handles=handles,
            training_num_steps=steps,
            training_lr_start=1e-3,
            training_lr_end=1e-3,
            training_le_coeff=1e-1,
            training_lo_coeff=1e6,
            training_log_every=max(1, steps // 10),
            normalize_for_training=True
        )
    
    logger.info("Training complete")
    return sim_obj


def setup_scene(sim_obj, gravity=(0, 9.8, 0), floor_height=-0.8, timestep=0.03):
    """Create scene and add physics."""
    scene = kal.physics.simplicits.SimplicitsScene()
    scene.max_newton_steps = 5
    scene.timestep = timestep
    scene.direct_solve = True
    
    obj_idx = scene.add_object(sim_obj)
    scene.set_scene_gravity(acc_gravity=torch.tensor(gravity))
    scene.set_scene_floor(floor_height=floor_height, floor_axis=1, floor_penalty=1000)
    
    return scene, obj_idx


def create_floor_mesh(floor_height=-0.8, size=5.0):
    """Create a simple floor plane mesh."""
    floor_verts = torch.tensor([
        [-size, floor_height, -size],
        [size, floor_height, -size],
        [size, floor_height, size],
        [-size, floor_height, size]
    ], device='cuda', dtype=torch.float32)
    
    floor_faces = torch.tensor([[0, 1, 2], [0, 2, 3]], device='cuda', dtype=torch.long)
    
    floor_mesh = kal.rep.SurfaceMesh(vertices=floor_verts, faces=floor_faces)
    return floor_mesh


def create_renderer(mesh, floor_height=-0.8, resolution=512):
    """Setup rendering functions with floor."""
    camera = kal.render.easy_render.default_camera(resolution).cuda()
    light_dir = kal.render.lighting.sg_direction_from_azimuth_elevation(1., 1.)
    lighting = kal.render.lighting.SgLightingParameters(amplitude=3., sharpness=5., direction=light_dir).cuda()
    
    floor_mesh = create_floor_mesh(floor_height)
    
    def render(in_cam):
        # Render object
        obj_res = kal.render.easy_render.render_mesh(in_cam, mesh, lighting=lighting)
        obj_img = obj_res[kal.render.easy_render.RenderPass.render]
        obj_depth = obj_res[kal.render.easy_render.RenderPass.depth]
        obj_mask = obj_res[kal.render.easy_render.RenderPass.face_idx] >= 0
        
        # Render floor
        floor_res = kal.render.easy_render.render_mesh(in_cam, floor_mesh, lighting=lighting)
        floor_img = floor_res[kal.render.easy_render.RenderPass.render] * 0.5 + 0.3  # Gray floor
        floor_depth = floor_res[kal.render.easy_render.RenderPass.depth]
        floor_mask = floor_res[kal.render.easy_render.RenderPass.face_idx] >= 0
        
        # Composite based on depth
        combined_img = torch.ones_like(obj_img)
        depth_mask = (obj_mask & floor_mask & (obj_depth < floor_depth)) | (obj_mask & ~floor_mask)
        combined_img[depth_mask] = obj_img[depth_mask]
        combined_img[floor_mask & ~depth_mask] = floor_img[floor_mask & ~depth_mask]
        
        img2 = torch.clamp(combined_img, 0, 1)[0]
        return {"img": (img2 * 255.).to(torch.uint8)}
    
    def fast_render(in_cam, factor=8):
        lowres_cam = copy.deepcopy(in_cam)
        lowres_cam.width = in_cam.width // factor
        lowres_cam.height = in_cam.height // factor
        return render(lowres_cam)
    
    return camera, render, fast_render


logger.info("Helper functions defined")

In [4]:
# Main workflow
mesh_path = "assets/fox.obj"

# Load and sample
mesh, orig_vertices, pts = load_and_sample_mesh(mesh_path)

# Visualize sampled points
logger.info("Visualizing sampled points...")
plot = k3d.plot()
plot += k3d.points(pts.cpu().numpy(), point_size=0.01)
plot.display()

# Train simulation object
sim_obj = create_sim_object(pts)

# Setup scene
scene, obj_idx = setup_scene(sim_obj)

logger.info("Setup complete")

2025-09-30 01:41:36.551 | INFO     | __main__:load_and_sample_mesh:31 - Loaded mesh: 5002 vertices, 10000 faces
2025-09-30 01:41:36.614 | INFO     | __main__:load_and_sample_mesh:41 - Sampled 44104 interior points
2025-09-30 01:41:36.616 | INFO     | __main__:<module>:8 - Visualizing sampled points...


Output()

2025-09-30 01:41:36.664 | INFO     | __main__:create_sim_object:52 - Training Simplicits object: 10000 steps, 5 handles


Training:   0%|          | 0/10000 [00:00<?, ?step/s]

2025-09-30 01:42:54.639 | INFO     | __main__:create_sim_object:68 - Training complete
d:\code\PoIntInt\.venv\Lib\site-packages\warp\torch.py:262: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  if t.grad is not None:
d:\code\PoIntInt\.venv\Lib\site-packages\kaolin\physics\utils\warp_utilities.py:263: UserWarning: Sparse BSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (T

In [ ]:
mesh.vertices = orig_vertices

global sim_thread_open, sim_thread
sim_thread_open = False
sim_thread = None

def reset_simulation(visualizer):
    with visualizer.out:
        scene.reset_scene()
    mesh.vertices = scene.get_object_deformed_pts(obj_idx, orig_vertices)
    visualizer.render_update()

def run_sim():
    for _ in range(100):
        with visualizer.out:
            scene.run_sim_step()
            print(".", end="")
        mesh.vertices = scene.get_object_deformed_pts(obj_idx, orig_vertices)
        visualizer.render_update()

def start_simulation(b):
    global sim_thread_open, sim_thread
    with visualizer.out:
        if sim_thread_open:
            sim_thread.join()
            sim_thread_open = False
        sim_thread_open = True
        sim_thread = threading.Thread(target=run_sim, daemon=True)
        sim_thread.start()

camera, render, fast_render = create_renderer(mesh)
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=fast_render,
    max_fps=24, world_up_axis=1
)

buttons = [Button(description=x) for x in ['Run Sim', 'Reset']]
buttons[0].on_click(lambda e: start_simulation(e))
buttons[1].on_click(lambda e: reset_simulation(visualizer))

reset_simulation(visualizer)
display(HBox([visualizer.canvas, VBox(buttons)]), visualizer.out)

d:\code\PoIntInt\.venv\Lib\site-packages\kaolin\render\easy_render\mesh.py:282: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Cross.cpp:67.)
  im_bitangents = torch.nn.functional.normalize(torch.cross(im_tangents, im_base_normals), dim=-1)


Output()

....................................................................................................